# Backward induction

In [2]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. load packages 
import numpy as np

# c. local modules
from mytest import child_model

Estimate the model in order to extract the parameters and the transision probabilities 

In [86]:
# Step 1: Create an instance of child_model
model = child_model()

Define the utility

In [90]:
def util(N,d,model):
    return model.eta2*N + model.eta3*(N**2) + model.mu*d 

Use backwards induction

In [89]:
# Reset the parameters 
model.mu = -0.12
model.eta2 = 0.15
model.eta3 = -0.13

# Step 2: Initialize value function and policy
n = model.n
T = model.T
V = np.zeros([n, T + 1])
D = np.zeros([n, T + 1])
D[:,T] = 0
V[:,T] = util(np.arange(n), 0, model)

# Lists for storing values
value_0 = []
value_1 = []
maxV = []

# Step 3: Backward induction
for t in range(T-1, -1, -1):
    for nn in range(n):
        for d in range(2):
            if d == 0:
                value_0.append(util(nn, 0, model) + model.beta * V[:, t+1])
            if d == 1:
                value_1.append(util(nn, 1, model) + model.beta * V[:, t+1])
        maxV.append(np.maximum(value_0[-1], value_1[-1]))
        logsum = maxV[-1] + np.log(np.exp(value_0[-1] - maxV[-1]) + np.exp(value_1[-1] - maxV[-1]))
        V[nn, t] = model.P1[nn] @ logsum
        D[nn, t] = (1 - 1 / (1 + np.exp(V[nn, t+1] - V[nn, t]))).round(0)
